Black-Litterman
<small>

This notebook combines the separate strategies into a single portfolio via the Black-Litterman model.  
The implementation is based on the paper by Idzorek, Thomas M. "A Step-By-Step Guide to the Black-Litterman Model."
<small/>

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from utils.backtest import Backtest
from scipy.optimize import minimize

In [3]:
# Data preprocessing
price_df = pd.read_csv('data/price.csv', index_col='Date', parse_dates=True).shift(1)
market_cap_df = pd.read_csv('data/market_cap.csv', index_col='Date', parse_dates=True).shift(1)

rf_df = pd.read_csv('data/rf_rate.csv', index_col=0, parse_dates=True).shift(1)
rf_df = (rf_df/100 + 1)  ** (1/252) - 1

all_files = os.listdir("weight")
weight_files = [f for f in all_files if f.endswith(".csv")]
weight_dfs = [pd.read_csv(os.path.join("weight", file), index_col=0, parse_dates=True) for file in weight_files]

In [ ]:
# Black-Litterman model
return_df = price_df.pct_change(fill_method=None)
month_end = return_df.iloc[500:].groupby(pd.Grouper(freq='ME')).tail(1).index
month_end = [date for date in month_end if date in market_cap_df.index]

results = {}
pp = None
for date in tqdm(month_end):
    current_index = return_df.index.get_loc(date)

    # Implied Excess Equilibrium Return 
    r = return_df.iloc[current_index-500:current_index+1]
    excess_r = r - rf_df.loc[r.index].values

    w_mkt = market_cap_df.loc[date] / sum(market_cap_df.loc[date])
    sigma = r.cov().values
    lambd = np.dot(w_mkt, excess_r.mean()) / np.dot(np.dot(w_mkt, sigma), w_mkt)
    pi = lambd * np.dot(sigma, w_mkt).reshape(-1, 1)
    
    # Views
    P = []
    for df in weight_dfs:
        if date in df.index:
            w_vector = df.loc[date]
            m = (w_vector == 0).sum()
            w_vector[w_vector == 0] = -1/m
            P.append(w_vector)
    P = np.array(P)

    #Q = np.ones((P.shape[0],1))
    Q = (pp - pp.min())/ (pp.max() - pp.min()) if pp is not None and len(pp) == P.shape[0] else np.ones((P.shape[0],1))
    omega = np.diag([row @ sigma @ row for row in P])
    pp = np.expand_dims((P @ r.T + 1).prod(axis=1), axis=1)
    
    # Combined Return Distribution 
    tau = 0.05
    inv_sigma = np.linalg.inv(tau * sigma)
    inv_omega = np.linalg.inv(omega) 
    E_r = np.linalg.inv(inv_sigma + P.T @ inv_omega @ P) @ ((inv_sigma @ pi) + (P.T @ inv_omega @ Q))

    # Use the Combined Return Distribution in a mean-variance optimizer
    def sharpe_objective(w, mu, sigma, rf):
        w_mu = np.dot(w, mu)
        w_sigma_w = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
        return - (w_mu - rf) / w_sigma_w

    mu = E_r
    sigma = r.cov()
    rf = rf_df.loc[date]
    initial_guess = len(mu) * [1 / len(mu)]
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = [(0, 1) for _ in range(len(mu))]

    res = minimize(sharpe_objective, initial_guess, args=(mu, sigma, rf), method='SLSQP', bounds=bounds, constraints=constraints)
    results[date] = res.x

weight_df = pd.DataFrame.from_dict(results, orient='index', columns=return_df.columns)

backtest = Backtest(weight_df)
backtest.run()
backtest.show()
